<a href="https://colab.research.google.com/github/PercyAyimbilaNsolemna/Machine_Learning/blob/main/DataCamp_Neural_Network_Customer_Churn/DataCamp_Customer_Churn_Neural_Network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **DATACAMP COURSE_1: CLASSIFICATION**
This notebook implements the Logistic Regression used in the classification course to develop a model for the churn dataset.

This implementation uses the Neural Networks to develop the model for the problem.

The **Logistic Regression Algorithm** is used here because the target values of our dataset is binary. That is, either a **customer churns** **1** or a **customer does not churn** **0**


**A simple Neural Network** implementation to compare the performance of the normal Logistic Regression ML algorithm used in the previous notebook

In [1]:
#Imports numpy
import numpy as np
#Imports pandas
import pandas as pd
#Imports tensorflow as tf
import tensorflow as tf
#Import the train_test_split from sklearn model selection
from sklearn.model_selection import train_test_split
#Imports StandardScaler from scikitlearn preprocessing
from sklearn.preprocessing import StandardScaler
#Imports mean squared error from scikit learn
from sklearn.metrics import mean_squared_error
#Imports accuracy score from sklearn
from sklearn.metrics import accuracy_score
#Imports the Sequential model from tensorflow
from tensorflow.keras.models import Sequential
#Imporst the Dense layer from tensorflow layers
from tensorflow.keras.layers import Dense
#Imports OneHotEncoder from sklearn preprocessing
from sklearn.preprocessing import OneHotEncoder
#Imports warning
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# **IMPORT GOOGLE DRIVE**

This imports google drive to the project so that the dataset for the project can be accessed and used for training, cross-validating and testing our model

In [2]:
#Imports drive from google colab
from google.colab import drive

In [3]:
#Connects google drive to the google colab notebook
drive.mount('/content/drive')

Mounted at /content/drive


## **TELCO CUSTOMER CHURN**

The Telco Custome Churn dataset will be used to develop the ML model.

The dataset can be downloaded [here](https://www.kaggle.com/datasets/blastchar/telco-customer-churn)

Thanks to [kaggle](https://www.kaggle.com/)

In [4]:
#Loads the training data from the google drive using pandas
data = pd.read_csv('/content/drive/MyDrive/Datasets/Telco-Customer-Churn.csv')

## **DATA VISUALIZATION**

The dimensions or shape and some few training examples in our dataset will be explored.

In [5]:
#Outputs the type of data been used
print(f'The type of data used is \n {type(data)}')

The type of data used is 
 <class 'pandas.core.frame.DataFrame'>


In [6]:
#Output information about the data
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   gender            7043 non-null   object 
 2   SeniorCitizen     7043 non-null   int64  
 3   Partner           7043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   tenure            7043 non-null   int64  
 6   PhoneService      7043 non-null   object 
 7   MultipleLines     7043 non-null   object 
 8   InternetService   7043 non-null   object 
 9   OnlineSecurity    7043 non-null   object 
 10  OnlineBackup      7043 non-null   object 
 11  DeviceProtection  7043 non-null   object 
 12  TechSupport       7043 non-null   object 
 13  StreamingTV       7043 non-null   object 
 14  StreamingMovies   7043 non-null   object 
 15  Contract          7043 non-null   object 
 16  PaperlessBilling  7043 non-null   object 


In [7]:
#Outputs the shape or dimentions of the training data
print(f'The shape of the training data is {data.shape}')

The shape of the training data is (7043, 21)


In [8]:
#Outputs all the columns of the training data
#Includes both the features and the target
print(f'The training data has {len(data.columns)} columns') #data.columns for all the columns
print(f'\nThe columns are: ')
for column in data.columns:
  print(column)

The training data has 21 columns

The columns are: 
customerID
gender
SeniorCitizen
Partner
Dependents
tenure
PhoneService
MultipleLines
InternetService
OnlineSecurity
OnlineBackup
DeviceProtection
TechSupport
StreamingTV
StreamingMovies
Contract
PaperlessBilling
PaymentMethod
MonthlyCharges
TotalCharges
Churn


In [9]:
#Prints the first five rows of the training data
data.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [10]:
#Prints the last five rows of the data
data.tail()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
7038,6840-RESVB,Male,0,Yes,Yes,24,Yes,Yes,DSL,Yes,...,Yes,Yes,Yes,Yes,One year,Yes,Mailed check,84.80,1990.5,No
7039,2234-XADUH,Female,0,Yes,Yes,72,Yes,Yes,Fiber optic,No,...,Yes,No,Yes,Yes,One year,Yes,Credit card (automatic),103.20,7362.9,No
7040,4801-JZAZL,Female,0,Yes,Yes,11,No,No phone service,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.60,346.45,No
7041,8361-LTMKD,Male,1,Yes,No,4,Yes,Yes,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Mailed check,74.40,306.6,Yes
7042,3186-AJIEK,Male,0,No,No,66,Yes,No,Fiber optic,Yes,...,Yes,Yes,Yes,Yes,Two year,Yes,Bank transfer (automatic),105.65,6844.5,No


In [11]:
#Drops any row with a null value
data = data.dropna()
#data = data[~data['TotalCharges'] == ' ']
#Drops any row with an empty string ' ' as the value
data = data[~(data == ' ').any(axis=1)]
print(f'The shape of the training data after dropping all rows with an empty string is {data.shape}')

The shape of the training data after dropping all rows with an empty string is (7032, 21)


In [12]:
#Outputs the distinct or unique values in the churn column
print(f'The distinct columns in the {data.columns[20]} column are: \n {data[data.columns[20]].unique()} \n')
#Loops through the columns excluding the customer ID and displays the distinct values in each column
for column_index in range(1, len(data.columns)):
  print(f'The distinct values in the \033[1m {data.columns[column_index]} \033[0;0m column are: ')
  print(f'{data[data.columns[column_index]].unique()} \n')

The distinct columns in the Churn column are: 
 ['No' 'Yes'] 

The distinct values in the  gender  column are: 
['Female' 'Male'] 

The distinct values in the  SeniorCitizen  column are: 
[0 1] 

The distinct values in the  Partner  column are: 
['Yes' 'No'] 

The distinct values in the  Dependents  column are: 
['No' 'Yes'] 

The distinct values in the  tenure  column are: 
[ 1 34  2 45  8 22 10 28 62 13 16 58 49 25 69 52 71 21 12 30 47 72 17 27
  5 46 11 70 63 43 15 60 18 66  9  3 31 50 64 56  7 42 35 48 29 65 38 68
 32 55 37 36 41  6  4 33 67 23 57 61 14 20 53 40 59 24 44 19 54 51 26 39] 

The distinct values in the  PhoneService  column are: 
['No' 'Yes'] 

The distinct values in the  MultipleLines  column are: 
['No phone service' 'No' 'Yes'] 

The distinct values in the  InternetService  column are: 
['DSL' 'Fiber optic' 'No'] 

The distinct values in the  OnlineSecurity  column are: 
['No' 'Yes' 'No internet service'] 

The distinct values in the  OnlineBackup  column are: 
['Ye

## **ONE-HOT ENCODING OF CATEGORICAL FEATURES**

In this section all the features with categorical data(text) will be converted to a vector of 0s and 1s.

*For example*

1. The **Churn feature** with two disinct values **YES** and **NO** will be encoded as **1** for **YES**  and **0** for **NO**

2. The **Internet Service feature** with three distinct values **DSL**, **Fiber
Optic** and **No** will be decoded as a **1*3 vector**. <br>
A customer who has uses **DSL** will be encoded as **[1 0 0]**, a customer who uses **Fibre Optic** will be encoded as **[0 1 0]** and a customer who does **NOT** use any of the Internet Service Providers will be encoded as **[0 0 1]**.

The coilumns to be transformed are listed below
* **Gender**
* **Partner**
* **Dependents**
* **Phone Service**
* **Multiple Lines**
* **Internet Service**
* **Online Security**
* **Online Backup**
* **Device Protection**
* **Tech Support**
* **Streaming TV**
* **Streaming Movies**
* **Contract**
* **Paperless Billing**
* **Payment method**
* **Total Charges**
* **Churn**

In [13]:
#Drops the customerID column from the data
data = data.drop(['customerID'], axis=1)

In [14]:
#Cast the Total charges column from object to float to a float
data['TotalCharges'] = pd.to_numeric(data['TotalCharges'])
#Displays the data types held by each of the columns in the data
data.dtypes

gender               object
SeniorCitizen         int64
Partner              object
Dependents           object
tenure                int64
PhoneService         object
MultipleLines        object
InternetService      object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract             object
PaperlessBilling     object
PaymentMethod        object
MonthlyCharges      float64
TotalCharges        float64
Churn                object
dtype: object

In [15]:
#Creates an object from the OneHotEncoding class
oneHotEncoder = OneHotEncoder()


In [16]:
#Use the fit_transform method in OneHotEncoder class to convert the categorical data into numerical data
transformed_features = oneHotEncoder.fit_transform(data[data.select_dtypes(include=['object']).columns]).toarray()
print(f'The shape of the transformed encoded features is {transformed_features.shape}')
#Converts the transformed features from float to int
transformed_features = transformed_features.astype(int)
#Outputs the one hot encoded features
print(transformed_features)

The shape of the transformed encoded features is (7032, 43)
[[1 0 0 ... 0 1 0]
 [0 1 1 ... 1 1 0]
 [0 1 1 ... 1 0 1]
 ...
 [1 0 0 ... 0 1 0]
 [0 1 0 ... 1 0 1]
 [0 1 1 ... 0 1 0]]


In [17]:
#Extracts the one hot encoded features
encoded_features = oneHotEncoder.get_feature_names_out()
#Outputs the one hot encoded features
print(encoded_features)
print(f'\nThe total number of encoded features are {len(encoded_features)}')

['gender_Female' 'gender_Male' 'Partner_No' 'Partner_Yes' 'Dependents_No'
 'Dependents_Yes' 'PhoneService_No' 'PhoneService_Yes' 'MultipleLines_No'
 'MultipleLines_No phone service' 'MultipleLines_Yes'
 'InternetService_DSL' 'InternetService_Fiber optic' 'InternetService_No'
 'OnlineSecurity_No' 'OnlineSecurity_No internet service'
 'OnlineSecurity_Yes' 'OnlineBackup_No' 'OnlineBackup_No internet service'
 'OnlineBackup_Yes' 'DeviceProtection_No'
 'DeviceProtection_No internet service' 'DeviceProtection_Yes'
 'TechSupport_No' 'TechSupport_No internet service' 'TechSupport_Yes'
 'StreamingTV_No' 'StreamingTV_No internet service' 'StreamingTV_Yes'
 'StreamingMovies_No' 'StreamingMovies_No internet service'
 'StreamingMovies_Yes' 'Contract_Month-to-month' 'Contract_One year'
 'Contract_Two year' 'PaperlessBilling_No' 'PaperlessBilling_Yes'
 'PaymentMethod_Bank transfer (automatic)'
 'PaymentMethod_Credit card (automatic)' 'PaymentMethod_Electronic check'
 'PaymentMethod_Mailed check' 'Chu

In [18]:
#Creating a DataFrame for the encoded features
one_hot_encoded_features = pd.DataFrame(transformed_features, columns = encoded_features)
print(one_hot_encoded_features)

      gender_Female  gender_Male  Partner_No  Partner_Yes  Dependents_No  \
0                 1            0           0            1              1   
1                 0            1           1            0              1   
2                 0            1           1            0              1   
3                 0            1           1            0              1   
4                 1            0           1            0              1   
...             ...          ...         ...          ...            ...   
7027              0            1           0            1              0   
7028              1            0           0            1              0   
7029              1            0           0            1              0   
7030              0            1           0            1              1   
7031              0            1           1            0              1   

      Dependents_Yes  PhoneService_No  PhoneService_Yes  MultipleLines_No  \
0         

In [19]:
#Outputs the head of the one hot encodfed features
one_hot_encoded_features.head()

,gender_Female,gender_Male,Partner_No,Partner_Yes,Dependents_No,Dependents_Yes,PhoneService_No,PhoneService_Yes,MultipleLines_No,MultipleLines_No phone service,...,Contract_One year,Contract_Two year,PaperlessBilling_No,PaperlessBilling_Yes,PaymentMethod_Bank transfer (automatic),PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check,Churn_No,Churn_Yes
0,1,0,0,1,1,0,1,0,0,1,...,0,0,0,1,0,0,1,0,1,0
1,0,1,1,0,1,0,0,1,1,0,...,1,0,1,0,0,0,0,1,1,0
2,0,1,1,0,1,0,0,1,1,0,...,0,0,0,1,0,0,0,1,0,1
3,0,1,1,0,1,0,1,0,0,1,...,1,0,1,0,1,0,0,0,1,0
4,1,0,1,0,1,0,0,1,1,0,...,0,0,0,1,0,0,1,0,0,1


In [20]:
#Outputs the tail of the one hot encoded features
one_hot_encoded_features.tail()

,gender_Female,gender_Male,Partner_No,Partner_Yes,Dependents_No,Dependents_Yes,PhoneService_No,PhoneService_Yes,MultipleLines_No,MultipleLines_No phone service,...,Contract_One year,Contract_Two year,PaperlessBilling_No,PaperlessBilling_Yes,PaymentMethod_Bank transfer (automatic),PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check,Churn_No,Churn_Yes
7027,0,1,0,1,0,1,0,1,0,0,...,1,0,0,1,0,0,0,1,1,0
7028,1,0,0,1,0,1,0,1,0,0,...,1,0,0,1,0,1,0,0,1,0
7029,1,0,0,1,0,1,1,0,0,1,...,0,0,0,1,0,0,1,0,1,0
7030,0,1,0,1,1,0,0,1,0,0,...,0,0,0,1,0,0,0,1,0,1
7031,0,1,1,0,1,0,0,1,1,0,...,0,1,0,1,1,0,0,0,1,0


In [21]:
data.tail()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
7038,Male,0,Yes,Yes,24,Yes,Yes,DSL,Yes,No,Yes,Yes,Yes,Yes,One year,Yes,Mailed check,84.80,1990.50,No
7039,Female,0,Yes,Yes,72,Yes,Yes,Fiber optic,No,Yes,Yes,No,Yes,Yes,One year,Yes,Credit card (automatic),103.20,7362.90,No
7040,Female,0,Yes,Yes,11,No,No phone service,DSL,Yes,No,No,No,No,No,Month-to-month,Yes,Electronic check,29.60,346.45,No
7041,Male,1,Yes,No,4,Yes,Yes,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Mailed check,74.40,306.60,Yes
7042,Male,0,No,No,66,Yes,No,Fiber optic,Yes,No,Yes,Yes,Yes,Yes,Two year,Yes,Bank transfer (automatic),105.65,6844.50,No


In [22]:
# Reset indices to ensure alignment before concatinating to prevent NA in some of the cells
data.reset_index(drop=True, inplace=True)
one_hot_encoded_features.reset_index(drop=True, inplace=True)

print(data.shape)
print(one_hot_encoded_features.shape)

(7032, 20)
(7032, 43)


In [23]:
#Concatenates the one hot encoded features to the main training data
data_new = pd.concat([data, one_hot_encoded_features], axis=1)
data_new

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,...,Contract_One year,Contract_Two year,PaperlessBilling_No,PaperlessBilling_Yes,PaymentMethod_Bank transfer (automatic),PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check,Churn_No,Churn_Yes
0,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,...,0,0,0,1,0,0,1,0,1,0
1,Male,0,No,No,34,Yes,No,DSL,Yes,No,...,1,0,1,0,0,0,0,1,1,0
2,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,...,0,0,0,1,0,0,0,1,0,1
3,Male,0,No,No,45,No,No phone service,DSL,Yes,No,...,1,0,1,0,1,0,0,0,1,0
4,Female,0,No,No,2,Yes,No,Fiber optic,No,No,...,0,0,0,1,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7027,Male,0,Yes,Yes,24,Yes,Yes,DSL,Yes,No,...,1,0,0,1,0,0,0,1,1,0
7028,Female,0,Yes,Yes,72,Yes,Yes,Fiber optic,No,Yes,...,1,0,0,1,0,1,0,0,1,0
7029,Female,0,Yes,Yes,11,No,No phone service,DSL,Yes,No,...,0,0,0,1,0,0,1,0,1,0
7030,Male,1,Yes,No,4,Yes,Yes,Fiber optic,No,No,...,0,0,0,1,0,0,0,1,0,1


In [24]:
#Outputs the shape of the new DataFrame, which has the original features excluding the cunstomerID and the one hot encoded features
print(f'The shape of the new DataFrame is {data_new.shape}')

The shape of the new DataFrame is (7032, 63)


In [25]:
#Outputs the features of the new training data
print(data_new.columns)

Index(['gender', 'SeniorCitizen', 'Partner', 'Dependents', 'tenure',
       'PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity',
       'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV',
       'StreamingMovies', 'Contract', 'PaperlessBilling', 'PaymentMethod',
       'MonthlyCharges', 'TotalCharges', 'Churn', 'gender_Female',
       'gender_Male', 'Partner_No', 'Partner_Yes', 'Dependents_No',
       'Dependents_Yes', 'PhoneService_No', 'PhoneService_Yes',
       'MultipleLines_No', 'MultipleLines_No phone service',
       'MultipleLines_Yes', 'InternetService_DSL',
       'InternetService_Fiber optic', 'InternetService_No',
       'OnlineSecurity_No', 'OnlineSecurity_No internet service',
       'OnlineSecurity_Yes', 'OnlineBackup_No',
       'OnlineBackup_No internet service', 'OnlineBackup_Yes',
       'DeviceProtection_No', 'DeviceProtection_No internet service',
       'DeviceProtection_Yes', 'TechSupport_No',
       'TechSupport_No internet service

In [26]:
#Outputs the datatype of each of the features to be sure all are in discrete
data_new.dtypes

gender                                   object
SeniorCitizen                             int64
Partner                                  object
Dependents                               object
tenure                                    int64
                                          ...  
PaymentMethod_Credit card (automatic)     int64
PaymentMethod_Electronic check            int64
PaymentMethod_Mailed check                int64
Churn_No                                  int64
Churn_Yes                                 int64
Length: 63, dtype: object

In [27]:
#Outputs the head of the concatenated training data
data_new.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,...,Contract_One year,Contract_Two year,PaperlessBilling_No,PaperlessBilling_Yes,PaymentMethod_Bank transfer (automatic),PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check,Churn_No,Churn_Yes
0,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,...,0,0,0,1,0,0,1,0,1,0
1,Male,0,No,No,34,Yes,No,DSL,Yes,No,...,1,0,1,0,0,0,0,1,1,0
2,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,...,0,0,0,1,0,0,0,1,0,1
3,Male,0,No,No,45,No,No phone service,DSL,Yes,No,...,1,0,1,0,1,0,0,0,1,0
4,Female,0,No,No,2,Yes,No,Fiber optic,No,No,...,0,0,0,1,0,0,1,0,0,1


In [28]:
#Outputs the tail of the new DataFrame
data_new.tail()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,...,Contract_One year,Contract_Two year,PaperlessBilling_No,PaperlessBilling_Yes,PaymentMethod_Bank transfer (automatic),PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check,Churn_No,Churn_Yes
7027,Male,0,Yes,Yes,24,Yes,Yes,DSL,Yes,No,...,1,0,0,1,0,0,0,1,1,0
7028,Female,0,Yes,Yes,72,Yes,Yes,Fiber optic,No,Yes,...,1,0,0,1,0,1,0,0,1,0
7029,Female,0,Yes,Yes,11,No,No phone service,DSL,Yes,No,...,0,0,0,1,0,0,1,0,1,0
7030,Male,1,Yes,No,4,Yes,Yes,Fiber optic,No,No,...,0,0,0,1,0,0,0,1,0,1
7031,Male,0,No,No,66,Yes,No,Fiber optic,Yes,No,...,0,1,0,1,1,0,0,0,1,0


In [29]:
data.tail()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
7027,Male,0,Yes,Yes,24,Yes,Yes,DSL,Yes,No,Yes,Yes,Yes,Yes,One year,Yes,Mailed check,84.80,1990.50,No
7028,Female,0,Yes,Yes,72,Yes,Yes,Fiber optic,No,Yes,Yes,No,Yes,Yes,One year,Yes,Credit card (automatic),103.20,7362.90,No
7029,Female,0,Yes,Yes,11,No,No phone service,DSL,Yes,No,No,No,No,No,Month-to-month,Yes,Electronic check,29.60,346.45,No
7030,Male,1,Yes,No,4,Yes,Yes,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Mailed check,74.40,306.60,Yes
7031,Male,0,No,No,66,Yes,No,Fiber optic,Yes,No,Yes,Yes,Yes,Yes,Two year,Yes,Bank transfer (automatic),105.65,6844.50,No


## **FEATURE ENGINEERING**

This section tackles selecting relevant features in the concatenated dataFrame for training the Machine Learning Model.

**Gender:** The Gender feature will be retained with 1 referring to a customer being a male and a zero representing a customer beinng a female. The gender_male and gender_female columns will be removed. Conincisely, the **male column** will be converted to the **gender column**.

**Partner:** The Partner feature will be ratained with 1 being a customer with a partner **Yes** and 0 for a customer without a partner **No**

**Dependent:** A customer **with a dependent** will be saved as **1** and a customer **without a dependent** will be saved as **0**.

**Phone Service:** Customers **with Phone Service** will be saved as **1** whiles those **without a Phone Service** will be saved as **0**.

**Paperless Billing:** Customers with **Paperless Billing** will be saved as **1** whiles those **without Paperless Billing** will be saved as **0**.

**Churn:** Customers that churned will be saved as 1 and customers that did not churn will be saved as 0.

**Multiple Lines:** The **Multiple Lines** column will be **removed** leaving the **one hot encoded fields.**

The following listed columns will be removed leaving the one hot encoded features
**Internet Service**, **Online Security**, **Online BackUp**, **Device Protection**, **Tech Support**, **Streaming TV**, **Streaming Movies**, **Contract**, **Payment Method**.

In [30]:
print(data_new.columns)

Index(['gender', 'SeniorCitizen', 'Partner', 'Dependents', 'tenure',
       'PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity',
       'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV',
       'StreamingMovies', 'Contract', 'PaperlessBilling', 'PaymentMethod',
       'MonthlyCharges', 'TotalCharges', 'Churn', 'gender_Female',
       'gender_Male', 'Partner_No', 'Partner_Yes', 'Dependents_No',
       'Dependents_Yes', 'PhoneService_No', 'PhoneService_Yes',
       'MultipleLines_No', 'MultipleLines_No phone service',
       'MultipleLines_Yes', 'InternetService_DSL',
       'InternetService_Fiber optic', 'InternetService_No',
       'OnlineSecurity_No', 'OnlineSecurity_No internet service',
       'OnlineSecurity_Yes', 'OnlineBackup_No',
       'OnlineBackup_No internet service', 'OnlineBackup_Yes',
       'DeviceProtection_No', 'DeviceProtection_No internet service',
       'DeviceProtection_Yes', 'TechSupport_No',
       'TechSupport_No internet service

In [31]:
#Drop all the aforementioned fields
#Drops the columns inplace. That is no new DataFrame is created
data_new.drop(['gender', 'gender_Female', 'Partner', 'Partner_No', 'Dependents', 'Dependents_No', 'PhoneService', 'PhoneService_No', 'PaperlessBilling',
               'PaperlessBilling_No','Churn','Churn_No', 'MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport',
               'StreamingTV', 'StreamingMovies', 'Contract', 'PaymentMethod'], axis = 1, inplace = True)

In [32]:
#Outputs the columns in the data_new DataFrame
print(data_new.columns)

Index(['SeniorCitizen', 'tenure', 'MonthlyCharges', 'TotalCharges',
       'gender_Male', 'Partner_Yes', 'Dependents_Yes', 'PhoneService_Yes',
       'MultipleLines_No', 'MultipleLines_No phone service',
       'MultipleLines_Yes', 'InternetService_DSL',
       'InternetService_Fiber optic', 'InternetService_No',
       'OnlineSecurity_No', 'OnlineSecurity_No internet service',
       'OnlineSecurity_Yes', 'OnlineBackup_No',
       'OnlineBackup_No internet service', 'OnlineBackup_Yes',
       'DeviceProtection_No', 'DeviceProtection_No internet service',
       'DeviceProtection_Yes', 'TechSupport_No',
       'TechSupport_No internet service', 'TechSupport_Yes', 'StreamingTV_No',
       'StreamingTV_No internet service', 'StreamingTV_Yes',
       'StreamingMovies_No', 'StreamingMovies_No internet service',
       'StreamingMovies_Yes', 'Contract_Month-to-month', 'Contract_One year',
       'Contract_Two year', 'PaperlessBilling_Yes',
       'PaymentMethod_Bank transfer (automatic)',
 

In [33]:
#Renames the columns that had binary labels to their original column names
#That is, Gender, Partner, Dependents, Phone Service, Paperless Billing, Churn
data_new.rename(columns = {'gender_Male': 'Gender', 'Partner_Yes': 'Partner', 'Dependents_Yes': 'Dependents', 'PhoneService_Yes': 'PhoneService',
                           'PaperlessBilling_Yes': 'PaperlessBilling', 'Churn_Yes': 'Churn', 'tenure': 'Tenure'}, inplace = True)

In [34]:
#Outputs the columns of the data_new DataFrame
print(data_new.columns)

Index(['SeniorCitizen', 'Tenure', 'MonthlyCharges', 'TotalCharges', 'Gender',
       'Partner', 'Dependents', 'PhoneService', 'MultipleLines_No',
       'MultipleLines_No phone service', 'MultipleLines_Yes',
       'InternetService_DSL', 'InternetService_Fiber optic',
       'InternetService_No', 'OnlineSecurity_No',
       'OnlineSecurity_No internet service', 'OnlineSecurity_Yes',
       'OnlineBackup_No', 'OnlineBackup_No internet service',
       'OnlineBackup_Yes', 'DeviceProtection_No',
       'DeviceProtection_No internet service', 'DeviceProtection_Yes',
       'TechSupport_No', 'TechSupport_No internet service', 'TechSupport_Yes',
       'StreamingTV_No', 'StreamingTV_No internet service', 'StreamingTV_Yes',
       'StreamingMovies_No', 'StreamingMovies_No internet service',
       'StreamingMovies_Yes', 'Contract_Month-to-month', 'Contract_One year',
       'Contract_Two year', 'PaperlessBilling',
       'PaymentMethod_Bank transfer (automatic)',
       'PaymentMethod_Credit 

In [35]:
#Outputs the head of the data_new
data_new.head()

,SeniorCitizen,Tenure,MonthlyCharges,TotalCharges,Gender,Partner,Dependents,PhoneService,MultipleLines_No,MultipleLines_No phone service,...,StreamingMovies_Yes,Contract_Month-to-month,Contract_One year,Contract_Two year,PaperlessBilling,PaymentMethod_Bank transfer (automatic),PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check,Churn
0,0,1,29.85,29.85,0,1,0,0,0,1,...,0,1,0,0,1,0,0,1,0,0
1,0,34,56.95,1889.50,1,0,0,1,1,0,...,0,0,1,0,0,0,0,0,1,0
2,0,2,53.85,108.15,1,0,0,1,1,0,...,0,1,0,0,1,0,0,0,1,1
3,0,45,42.30,1840.75,1,0,0,0,0,1,...,0,0,1,0,0,1,0,0,0,0
4,0,2,70.70,151.65,0,0,0,1,1,0,...,0,1,0,0,1,0,0,1,0,1


In [36]:
#Outputs the tail of the data_new
data_new.tail()

,SeniorCitizen,Tenure,MonthlyCharges,TotalCharges,Gender,Partner,Dependents,PhoneService,MultipleLines_No,MultipleLines_No phone service,...,StreamingMovies_Yes,Contract_Month-to-month,Contract_One year,Contract_Two year,PaperlessBilling,PaymentMethod_Bank transfer (automatic),PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check,Churn
7027,0,24,84.80,1990.50,1,1,1,1,0,0,...,1,0,1,0,1,0,0,0,1,0
7028,0,72,103.20,7362.90,0,1,1,1,0,0,...,1,0,1,0,1,0,1,0,0,0
7029,0,11,29.60,346.45,0,1,1,0,0,1,...,0,1,0,0,1,0,0,1,0,0
7030,1,4,74.40,306.60,1,1,0,1,0,0,...,0,1,0,0,1,0,0,0,1,1
7031,0,66,105.65,6844.50,1,0,0,1,1,0,...,1,0,0,1,1,1,0,0,0,0


## **SPLITTING DATA INTO FEATURES AND TARGET VALUES**

This section splits the training data into training features and target values that is X and y

In [37]:
#Features and converts it to a numpy array
X = data_new.drop(['Churn'], axis=1).values
print(f'The shape of the training data is {X.shape} \n')
print(type(X))

#Target value and converts it to a numpy array
y = data_new['Churn'].values
print(f'The shape of the target variable is {y.shape}')

The shape of the training data is (7032, 40) 

<class 'numpy.ndarray'>
The shape of the target variable is (7032,)


## **SPLITTING THE TRAINING DATA**

This section splits the **data_new into training, cross-validation and testing** data using **train_test_split from scikit learn model selection**

In [38]:
#Splits the data into training, cross-validation and testing
# Get 60% of the dataset as the training set. Put the remaining 40% in temporary variables: x_ and y_.
X_train, X_, y_train, y_ = train_test_split(X, y, train_size=0.6, random_state=42, stratify=y)
# Split the 40% subset above into two: one half for cross validation and the other for the test set
X_test, X_cv, y_test, y_cv = train_test_split(X_, y_, test_size=0.5, random_state=42, stratify=y_)
#Deletes the temporary variables
del X_, y_

print(f'The shape of the training set (input) is {X_train.shape}')
print(f'The shape of the training set (target) is {y_train.shape}\n')
print(f'The shape of the cross validation set (input) is {X_cv.shape}')
print(f'The shape of the cross validation set (target) is {y_cv.shape}\n')
print(f'The shape of the test set (input) is {X_test.shape}')
print(f'The shape of the test set (target) is {y_test.shape}')

The shape of the training set (input) is (4219, 40)
The shape of the training set (target) is (4219,)

The shape of the cross validation set (input) is (1407, 40)
The shape of the cross validation set (target) is (1407,)

The shape of the test set (input) is (1406, 40)
The shape of the test set (target) is (1406,)


## **FEATURE SCALING**

This section focuses on feature scaling which enables the model to **converge faster**.

Feature scaling ensures that the features have a **variance and standard deviation of zero**.

**Feature scaling = (feature - mean) / standard_deviation**

In [39]:
#Instantiate the Standard Scaler from scikitlearn
standardScaler = StandardScaler()

#Scales the training set
X_train_scaled = standardScaler.fit_transform(X_train)

#Outputs the calculated mean of the training data
print(f'The calculated mean of the training data for each feature is {standardScaler.mean_}\n')

#Outputs the min and max values in the unscaled X_train data
print(f'The maximum value in the X_train unscaled is {X_train.max()}')
print(f'The minimum value in the X_train unscaled is {X_train.min()}\n')

#Outputs the min and max of the values in the scaled X_train data
print(f'The maximum value in the scaled X_train is {X_train_scaled.max()}')
print(f'The minimum value in the scaled X_train is {X_train_scaled.min()}')

The calculated mean of the training data for each feature is [1.62834795e-01 3.25005926e+01 6.48069803e+01 2.29445905e+03
 5.02962787e-01 4.81867741e-01 2.97700877e-01 9.04716758e-01
 4.80682626e-01 9.52832425e-02 4.24034131e-01 3.41550130e-01
 4.40151695e-01 2.18298175e-01 4.90874615e-01 2.18298175e-01
 2.90827210e-01 4.36359327e-01 2.18298175e-01 3.45342498e-01
 4.36122304e-01 2.18298175e-01 3.45579521e-01 4.87556293e-01
 2.18298175e-01 2.94145532e-01 3.95354349e-01 2.18298175e-01
 3.86347476e-01 3.97961602e-01 2.18298175e-01 3.83740223e-01
 5.47997156e-01 2.04313818e-01 2.47689026e-01 5.93268547e-01
 2.21616497e-01 2.17350083e-01 3.33728372e-01 2.27305049e-01]

The maximum value in the X_train unscaled is 8684.8
The minimum value in the X_train unscaled is 0.0

The maximum value in the scaled X_train is 3.081399824044562
The minimum value in the scaled X_train is -3.081399824044562


In [40]:
#Scales the X_cv using the mean and stadard deviation used to scale the training data
X_cv_scaled = standardScaler.fit_transform(X_cv)

#Scales the X_test using the mean and standard deviation used to scale the training data
X_test_scaled = standardScaler.fit_transform(X_test)

## **DEVELOPING THE MODEL**

This section deals with implementing different **Sequential Neural Network architectures**. These architectures will be trained using the training set whiles their performance are assessed using the cross-validation set.

In addition, **regularization** will be emplored where needed to solve overfitting as well as using **Adams Optimizer** a much improved version of the learning rate used in training an Machine Leearning model.

In [41]:
#A dictionary to storer the train accuracies
train_accuracies = {}

#A dictionary to store the cross-validation accuracies
cross_validation_accuracies = {}

## **DEFINING THE MODELS**

This defines different models with different layers but the same activation function for the hidden layers for the customer churn problem.

These models will be trained on the training data set and based on the total cost(mean squared error) of the training data set and the cross validation set a suitable model will be chosen.

 The model will be fine-tuned to get a well suitable parameters that fit the data well and generalizes very well (just right model)

In [42]:
#Defines the models with the same activation function 'relu' in the hidden layers
#Uses the linear activation function in the output layer
#The from_logits parameter in the BinaryCrossEntropy will be set to True to prevent run off errors
#That also specifies that the output layer is a logistic regression

models = [
    Sequential(
        [
            Dense(units=25, activation='relu'),
            Dense(units=15, activation='relu'),
            Dense(units=1, activation='linear')
        ], name = 'Model_1'
    ),

    Sequential(
        [
            Dense(units=20, activation='relu'),
            Dense(units=30, activation='relu'),
            Dense(units=15, activation='relu'),
            Dense(units=10, activation='relu'),
            Dense(units=1, activation='linear')
        ], name = 'Model_2'
    ),

    Sequential(
        [
            Dense(units=5, activation='relu'),
            Dense(units=25, activation='relu'),
            Dense(units=1, activation='linear')
        ], name = 'Model_3'
    ),

    Sequential(
        [
            Dense(units=20, activation='relu'),
            Dense(units=30, activation='relu'),
            Dense(units=40, activation='relu'),
            Dense(units=1, activation='linear')
        ], name = 'Model_4'
    )
]

## **Trains the neural network**

* This trains the neural network using the training data and the data specified below

* Trains the neural network using the X_train_scaled dataset

* Trains the training data using the models specified in the above code

* Validates each of the models using the cross validation dataset to select the best model for fine-tuning the model

* Specifies and imports the **BinaryCrossEntropy cost function** to be used to train the neural network. Since the output layer is a **logistic regression (binary layer)**, the from_logits parameter in the function will be set to true.

* The **from_logits set to True** shows that the last layer in the neural network is **multilable classification** hence tensorFlow uses a more error tolerating logistic **regression function as the activation function** for that layer, **reducing run-off errors**

* Specifies the **Adaptive Gradient Descent (Adam) optimizer** to be used during training to enable gradiesnt descent get to the local minimum faster which inturn speeds up the training process

* Specifies the **epochs (number of iterations)** during training for each of the models

* Verbose set to 1 or 2 shows an animated bar of the epoches during training

In [43]:
#Imports BinaryCrossEntropy cost function from tensorflow keras losses
from tensorflow.keras.losses import BinaryCrossentropy

#Imports the Adaptive Gradient Descent (Adam) optimizer from tensorflow
from tensorflow.keras.optimizers import Adam

In [44]:
#Iterates through each of the models, train the neural networks and outpust the mean squared error for both
#The training data nd the cross validation data
for model in models:
  model.compile(
      loss = BinaryCrossentropy(from_logits=True),
      optimizer = Adam(0.001)
  )

  print(f'Training: {model.name}...')

  model.fit(
      X_train_scaled, y_train,
      epochs=100,
      verbose=0
      )

  print('Done!! \n')

  #Use the trained model to predict the X_train scaled data
  prediction_train = model.predict(X_train_scaled)
  #Runs the predicted values through a sigmoid from since the output is binary. The output is the probability of the output being 0 or 1
  prediction_prob_train = tf.nn.sigmoid(prediction_train).numpy()
  #Converts each predicted output as 0 or 1 using the calculated probabilities
  prediction_train = (prediction_prob_train >= 0.5).astype(int)
  #Calculates the accuracy score
  accuracy_score_train = accuracy_score(y_train, prediction_train)

  #Record the accuracy score to the train accuracies
  train_accuracies[model.name] = accuracy_score_train

  #Use the trained model to predict the X_cv scaled data
  prediction_cv = model.predict(X_cv_scaled)
  #Runs the predicted values through a sigmoid from since the output is binary. The output is the probability of the output being 0 or 1
  prediction_prob_cv = tf.nn.sigmoid(prediction_cv).numpy()
  #Converts each predicted output as 0 or 1 using the calculated probabilities
  prediction_cv = (prediction_prob_cv >= 0.5).astype(int)
  #Calculates the accuracy score
  accuracy_score_cv = accuracy_score(y_cv, prediction_cv)

  #Record the accuracy score to teh c accuracies
  cross_validation_accuracies[model.name] = accuracy_score_cv


Training: Model_1...
Done!! 

44/44 [==============================] - 0s 2ms/step
Training: Model_2...
Done!! 

44/44 [==============================] - 0s 2ms/step
Training: Model_3...
Done!! 

44/44 [==============================] - 0s 1ms/step
Training: Model_4...
Done!! 

44/44 [==============================] - 0s 1ms/step


In [45]:
#Outputs the training accuracy and cross validation for each of the models
print(f'Training accuracy:         {train_accuracies}')
print(f'Cross validation accuracy: {cross_validation_accuracies}')

Training accuracy:         {'Model_1': 0.8694003318321877, 'Model_2': 0.9224934818677412, 'Model_3': 0.8136999288931026, 'Model_4': 0.9160938611045272}
Cross validation accuracy: {'Model_1': 0.7683013503909026, 'Model_2': 0.7334754797441365, 'Model_3': 0.7953091684434968, 'Model_4': 0.7448471926083866}


## **MODEL SELECTION**

All the models perform well with good training and cross-validation accuracy score but the **2nd model will be selected**.

Although the the 2nd model performs very well on the training data and generalize quite well on the cross-validation set which is a **clear sign of overfitting**.

The parameters of the 2nd model will be **regularized to penalize each of the paramaters to solve the overfitting problem to enable the model generalize pretty well**.

In [46]:
#List of regularized models
regularized_models = [
    Sequential(
        [
            Dense(units=20, activation='relu', kernel_regularizer = tf.keras.regularizers.l2(0.002)),
            Dense(units=30, activation='relu', kernel_regularizer = tf.keras.regularizers.l2(0.002)),
            Dense(units=15, activation='relu', kernel_regularizer = tf.keras.regularizers.l2(0.002)),
            Dense(units=10, activation='relu', kernel_regularizer = tf.keras.regularizers.l2(0.002)),
            Dense(units=1, activation='linear')
        ], name = 'Model_2_reg_1'
    ),

    Sequential(
        [
            Dense(units=20, activation='relu', kernel_regularizer = tf.keras.regularizers.l2(0.007)),
            Dense(units=30, activation='relu', kernel_regularizer = tf.keras.regularizers.l2(0.007)),
            Dense(units=15, activation='relu', kernel_regularizer = tf.keras.regularizers.l2(0.007)),
            Dense(units=10, activation='relu', kernel_regularizer = tf.keras.regularizers.l2(0.007)),
            Dense(units=1, activation='linear')
        ], name = 'Model_2_reg_2'
    ),

    Sequential(
        [
            Dense(units=20, activation='relu', kernel_regularizer = tf.keras.regularizers.l2(0.0004)),
            Dense(units=30, activation='relu', kernel_regularizer = tf.keras.regularizers.l2(0.0004)),
            Dense(units=15, activation='relu', kernel_regularizer = tf.keras.regularizers.l2(0.0004)),
            Dense(units=10, activation='relu', kernel_regularizer = tf.keras.regularizers.l2(0.0004)),
            Dense(units=1, activation='linear')
        ], name = 'Model_2_reg_3'
    )
]

In [47]:
#A dictionary to store the train accuracies of the regularized model
reg_train_accuracies = {}

#A dictionary to store the cross-validation accuracies of the regularized model
reg_cross_validation_accuracies = {}

In [48]:
#Iterates through each of the models, train the neural networks and outpust the mean squared error for both
#The training data nd the cross validation data
for model in regularized_models:
  model.compile(
      loss = BinaryCrossentropy(from_logits=True),
      optimizer = Adam(0.001)
  )

  print(f'Training: {model.name}...')

  model.fit(
      X_train_scaled, y_train,
      epochs=100,
      verbose=0
      )

  print('Done!! \n')

  #Use the trained model to predict the X_train scaled data
  prediction_train = model.predict(X_train_scaled)
  #Runs the predicted values through a sigmoid from since the output is binary. The output is the probability of the output being 0 or 1
  prediction_prob_train = tf.nn.sigmoid(prediction_train).numpy()
  #Converts each predicted output as 0 or 1 using the calculated probabilities
  prediction_train = (prediction_prob_train >= 0.5).astype(int)
  #Calculates the accuracy score
  accuracy_score_train = accuracy_score(y_train, prediction_train)

  #Record the accuracy score to the train accuracies
  reg_train_accuracies[model.name] = accuracy_score_train

  #Use the trained model to predict the X_cv scaled data
  prediction_cv = model.predict(X_cv_scaled)
  #Runs the predicted values through a sigmoid from since the output is binary. The output is the probability of the output being 0 or 1
  prediction_prob_cv = tf.nn.sigmoid(prediction_cv).numpy()
  #Converts each predicted output as 0 or 1 using the calculated probabilities
  prediction_cv = (prediction_prob_cv >= 0.5).astype(int)
  #Calculates the accuracy score
  accuracy_score_cv = accuracy_score(y_cv, prediction_cv)

  #Record the accuracy score to teh c accuracies
  reg_cross_validation_accuracies[model.name] = accuracy_score_cv


Training: Model_2_reg_1...
Done!! 

44/44 [==============================] - 0s 2ms/step
Training: Model_2_reg_2...
Done!! 

44/44 [==============================] - 0s 1ms/step
Training: Model_2_reg_3...
Done!! 

44/44 [==============================] - 0s 2ms/step


In [49]:
#Outputs the training accuracy and cross validation for each of the regularized models
print(f'Reg Training accuracy:         {reg_train_accuracies}')
print(f'Reg Cross validation accuracy: {reg_cross_validation_accuracies}')

Reg Training accuracy:         {'Model_2_reg_1': 0.8563640673145295, 'Model_2_reg_2': 0.8091964920597298, 'Model_2_reg_3': 0.8990282057359564}
Reg Cross validation accuracy: {'Model_2_reg_1': 0.7896233120113717, 'Model_2_reg_2': 0.7981520966595593, 'Model_2_reg_3': 0.7697228144989339}


## **REGULARIZED MODEL SELECTION**

For this regularized version of model 2. the second model will be selected since it has the greatest cross-validation accuracy score.

Model three in the prior defined models will be regularized to check how well it will also perform.

In [78]:
#Store the accuracies of the train and cross validation data sets
reg3_train_accuracies = {}

reg3_cross_validation_accuracies = {}

In [79]:
regularized_models_3 = [
     Sequential(
        [
            Dense(units=5, activation='relu', kernel_regularizer = tf.keras.regularizers.l2(0.02)),
            Dense(units=25, activation='relu', kernel_regularizer = tf.keras.regularizers.l2(0.02)),
            Dense(units=1, activation='linear')
        ], name = 'Model_3_regularized_1'
    ),

     Sequential(
        [
            Dense(units=5, activation='relu', kernel_regularizer = tf.keras.regularizers.l2(0.015)),
            Dense(units=25, activation='relu', kernel_regularizer = tf.keras.regularizers.l2(0.015)),
            Dense(units=1, activation='linear')
        ], name = 'Model_3_regularized_2'
    ),

     Sequential(
        [
            Dense(units=5, activation='relu', kernel_regularizer = tf.keras.regularizers.l2(0.016)),
            Dense(units=25, activation='relu', kernel_regularizer = tf.keras.regularizers.l2(0.016)),
            Dense(units=1, activation='linear')
        ], name = 'Model_3_regularized_3'
    ),
]

In [76]:
#Iterates through each of the models, train the neural networks and outpust the mean squared error for both
#The training data nd the cross validation data
for model in regularized_models_3:
  model.compile(
      loss = BinaryCrossentropy(from_logits=True),
      optimizer = Adam(0.001)
  )

  print(f'Training: {model.name}...')

  model.fit(
      X_train_scaled, y_train,
      epochs=100,
      verbose=0
      )

  print('Done!! \n')

  #Use the trained model to predict the X_train scaled data
  prediction_train = model.predict(X_train_scaled)
  #Runs the predicted values through a sigmoid from since the output is binary. The output is the probability of the output being 0 or 1
  prediction_prob_train = tf.nn.sigmoid(prediction_train).numpy()
  #Converts each predicted output as 0 or 1 using the calculated probabilities
  prediction_train = (prediction_prob_train >= 0.5).astype(int)
  #Calculates the accuracy score
  accuracy_score_train = accuracy_score(y_train, prediction_train)

  #Record the accuracy score to the train accuracies
  reg3_train_accuracies[model.name] = accuracy_score_train

  #Use the trained model to predict the X_cv scaled data
  prediction_cv = model.predict(X_cv_scaled)
  #Runs the predicted values through a sigmoid from since the output is binary. The output is the probability of the output being 0 or 1
  prediction_prob_cv = tf.nn.sigmoid(prediction_cv).numpy()
  #Converts each predicted output as 0 or 1 using the calculated probabilities
  prediction_cv = (prediction_prob_cv >= 0.5).astype(int)
  #Calculates the accuracy score
  accuracy_score_cv = accuracy_score(y_cv, prediction_cv)

  #Record the accuracy score to teh c accuracies
  reg3_cross_validation_accuracies[model.name] = accuracy_score_cv


Training: Model_3_regularized_1...
Done!! 

44/44 [==============================] - 0s 2ms/step
Training: Model_3_regularized_2...
Done!! 

44/44 [==============================] - 0s 1ms/step
Training: Model_3_regularized_3...
Done!! 

44/44 [==============================] - 0s 1ms/step


In [77]:
#Outputs the training accuracy and cross validation for each of the regularized models
print(f'Reg Training accuracy:         {reg3_train_accuracies}')
print(f'Reg Cross validation accuracy: {reg3_cross_validation_accuracies}')

Reg Training accuracy:         {'Model_3_regularized_1': 0.8103816070158806, 'Model_3_regularized_2': 0.8087224460772695, 'Model_3_regularized_3': 0.8089594690684997}
Reg Cross validation accuracy: {'Model_3_regularized_1': 0.7896233120113717, 'Model_3_regularized_2': 0.806680881307747, 'Model_3_regularized_3': 0.8002842928216063}


## **FINAL REGULARIZED MODEL SELECTION**

Upon finetuning and regularizing the 3rd and 2nd models, the 3rd model proved to perform pretty well as compared to the regularized versions of the 2nd model.

The 2nd model in the regularized model 3 has the best training and cross-validation accuracy score hence the basis for its selection.

**In conclusion, the best model for the customer churn problem is the 2nd regularized model in the regularized version of the 3rd model.**

In [81]:
selected_model = Sequential(
        [
            Dense(units=5, activation='relu', kernel_regularizer = tf.keras.regularizers.l2(0.015)),
            Dense(units=25, activation='relu', kernel_regularizer = tf.keras.regularizers.l2(0.015)),
            Dense(units=1, activation='linear')
        ], name = 'prefered_model'
    )

## **TRAINING THE SELECTED MODEL ON THE TRAINING DATA**

The selected model is trained using the scaled training dataset

In [82]:
#Compiles the selected model
selected_model.compile(
    loss = BinaryCrossentropy(from_logits = True),
    optimizer = Adam(0.001)
)

print(f'Training the {selected_model.name} ...')

#Fits the training data to the model
selected_model.fit(X_train_scaled, y_train,
                   epochs = 100,
                   verbose = 1
                   )

Training the prefered_model ...
Epoch 1/100
132/132 [==============================] - 4s 7ms/step - loss: 0.8988
Epoch 2/100
132/132 [==============================] - 1s 8ms/step - loss: 0.6739
Epoch 3/100
132/132 [==============================] - 1s 7ms/step - loss: 0.5300
Epoch 4/100
132/132 [==============================] - 1s 5ms/step - loss: 0.4759
Epoch 5/100
132/132 [==============================] - 0s 3ms/step - loss: 0.4609
Epoch 6/100
132/132 [==============================] - 0s 2ms/step - loss: 0.4528
Epoch 7/100
132/132 [==============================] - 0s 2ms/step - loss: 0.4483
Epoch 8/100
132/132 [==============================] - 0s 2ms/step - loss: 0.4451
Epoch 9/100
132/132 [==============================] - 0s 2ms/step - loss: 0.4433
Epoch 10/100
132/132 [==============================] - 0s 2ms/step - loss: 0.4425
Epoch 11/100
132/132 [==============================] - 0s 2ms/step - loss: 0.4405
Epoch 12/100
132/132 [==============================] - 0s 2ms/s

## **PREDICTING THE TEST SET**

This section deals with using the trained model to predict the test dataset to check how well the model will perform on unseen data.

This gives an overview of how well the model will be able to generalize

In [114]:
#Use the trained model to predict the X_test data
prediction_test = selected_model.predict(X_test_scaled)

#Runs the predicted values through a sigmoid from since the output is binary. The output is the probability of the output being 0 or 1
prediction_prob_test = tf.nn.sigmoid(prediction_test).numpy()
#print(prediction_prob_test)

#Converts each predicted output as 0 or 1 using the calculated probabilities
prediction_test = (prediction_prob_test >= 0.5).astype(int)
#print(prediction_test)

44/44 [==============================] - 0s 3ms/step


In [115]:
#Flattens the predicted values to a 1D array
prediction_test = prediction_test.flatten()
#Outputs the first 30 target values and the predicted values
print(f'Target values:    {y_test[:30]}')
print(f'Predicted values: {prediction_test[:30]}')

Target values:    [1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 1 0 0 1 0 0 0 0 0 1]
Predicted values: [1 1 0 0 1 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 1 0 0 1 0 0 1 0 0 0]


## **MODEL SUMMARY**

This section gives a detailed overview of the selected model. The number of neurons in each layer, the output features in each layer together with the bias and the shape of each layer.

The **Dense layer** was used in each of the layers of the Neural Network.

The first layer has **five different neurons**. The initial training data has a total of **40 features** therefore each neuron in the first layer will have 40 parameters **totalling 200**. Since there are five neurons, there will be **five distinct biases** for that layer summing up to **205**.

Same applies to the second and tyhe third layers.

The second layer has its previous layer that is the first layer having five distict activation energies, each with 5 features. 5 by the totalk number of neurons in the second layer sums up to 125. Adding the 25 generated bias for each neuron makes a total sum of 150 parameters at the second layer.

Finally, the third layer has its previous layer with 25 activation energies. Since the layer has only one neuron the total features will be 25 * 1 making 25 with the additional bias which will be generated by the neurol making the overall sum 26.

In [116]:
#Outputs the summary of the selected model
selected_model.summary()

Model: "prefered_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_114 (Dense)           (None, 5)                 205       
                                                                 
 dense_115 (Dense)           (None, 25)                150       
                                                                 
 dense_116 (Dense)           (None, 1)                 26        
                                                                 
Total params: 381 (1.49 KB)
Trainable params: 381 (1.49 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


## **EXAMINE MODEL PARAMETER SHAPE**

This outputs the shapes of the parameters that is the weight and bias at each layer of the neural network

In [117]:
#Extracts all the layers in the netywork
[layer1, layer2, layer3] = selected_model.layers

#Gets the weights and bias at each layer
for index, layer in enumerate(selected_model.layers):
  w_index, b_index = layer.get_weights()
  print(f'W{index+1} shape: {w_index.shape}, b{index+1} shape: {b_index.shape}')

W1 shape: (40, 5), b1 shape: (5,)
W2 shape: (5, 25), b2 shape: (25,)
W3 shape: (25, 1), b3 shape: (1,)


## **VISUALIZE THE PARAMETERS AT A SPECIFIED LAYER**

This section gives a clear idea of what happens at each layer. The weights and the bias at a specific layer is retrieved and printed to the console.

Feel free to change the layers to see the weights of each of the parameters at your specified layer.

In [118]:
print(selected_model.layers[0].weights)

[<tf.Variable 'dense_114/kernel:0' shape=(40, 5) dtype=float32, numpy=
array([[-1.71832536e-36,  1.01844513e-25, -6.35109618e-02,
         9.64467376e-02, -7.49144852e-02],
       [ 1.30296374e-34, -1.49036010e-30, -5.76269627e-02,
        -3.15249324e-01,  1.23650901e-01],
       [ 2.28650136e-34, -1.50027058e-34,  7.52205551e-02,
        -2.36531701e-02, -5.65018468e-02],
       [ 3.00870511e-34, -1.20953811e-34, -2.37290561e-02,
        -1.86208934e-01,  2.41640247e-02],
       [-2.80886212e-34,  2.62011691e-30,  7.08392039e-02,
        -8.13174471e-02, -7.30283484e-02],
       [ 3.16220321e-36, -8.48107137e-31,  4.90534566e-02,
        -2.47684252e-02,  3.26753296e-02],
       [ 2.03766252e-34,  1.01683223e-34, -3.80670279e-02,
        -3.39685678e-02,  4.47961316e-02],
       [-7.21994944e-35, -1.01048947e-33, -5.27454168e-02,
        -7.69893080e-03, -1.25724897e-02],
       [ 2.65865437e-34,  5.40986311e-30, -6.50243610e-02,
        -6.18397398e-03, -2.59036751e-04],
       [ 7.

## **MISCLASSIFIED TARGET VALUES**

This section outputs the number of predicted values that have been misclassified in addition to the indexes of each of the misclaasified outputs

In [119]:
#Misclassified values
misclassified_outputs = np.where(y_test != prediction_test)
print(f'The number of misclassified outputs are {len(misclassified_outputs[0])+1}')

The number of misclassified outputs are 293


In [120]:
#Outputs the indexes of the misclassified outputs
print(f'The indexes of the misclassified outputs are: \n{misclassified_outputs}')

The indexes of the misclassified outputs are: 
(array([   4,   10,   15,   26,   29,   32,   38,   41,   42,   50,   51,
         56,   57,   66,   67,   71,   73,   79,   82,   83,   86,   87,
         91,   93,  101,  104,  107,  110,  113,  115,  116,  117,  123,
        127,  128,  143,  144,  145,  149,  156,  157,  158,  166,  167,
        170,  173,  176,  180,  184,  186,  191,  200,  203,  205,  215,
        216,  217,  220,  221,  226,  233,  234,  237,  241,  243,  244,
        252,  262,  265,  269,  270,  278,  279,  284,  288,  295,  297,
        298,  302,  311,  315,  317,  318,  322,  323,  326,  335,  354,
        362,  369,  372,  379,  387,  397,  407,  411,  413,  423,  430,
        432,  439,  440,  441,  445,  448,  451,  452,  464,  468,  471,
        475,  494,  500,  501,  503,  511,  514,  521,  524,  535,  542,
        546,  550,  551,  561,  564,  567,  568,  575,  576,  585,  587,
        594,  598,  606,  607,  609,  614,  625,  630,  631,  639,  642,
   

## **MODEL ACCURACY**

This section calculates the models accuracy using the test set

In [124]:
#Model accuracy
accuracy_score_test = accuracy_score(y_test, prediction_test)
#print(accuracy_score_test)
accuracy_score_test = accuracy_score_test * 100
print(f'\nThe model has an accuracy score of {accuracy_score_test:.0f}%' )

0.7923186344238976

The model has an accuracy score of 79%


## **THANK YOU FOR YOUR PATIENCE AND TIME**